# 03-01 - Plugin

## Overview Plugin


## Environment

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.38.0"
#r "nuget: DotNetEnv, 3.1.1"

using System.IO;
using System.ComponentModel;
using DotNetEnv;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;

string configurationFile = "../config/config.env";

Env.Load(configurationFile);

string apiKey = Env.GetString("SK_OPENAI_APIKEY");
string endpoint = Env.GetString("SK_OPENAI_ENDPOINT");
string chatCompletionDeployment = Env.GetString("SK_OPENAI_CHATCOMPLETION_DEPLOYMENT_DEFAULT");

Console.WriteLine($"Configuration loaded...");

Installed Packages DotNetEnv, 3.1.1 Microsoft.SemanticKernel, 1.38.0

Configuration loaded...


## Define Plugin Functionality

In [ ]:
class CodeLibrary
{
    [KernelFunction("get_sport_event_winner")]
    //Description is just necessary if function name is not self-explanatory
    [Description("Get the winner of a sport event. The event is identified by the sport event name and the year.")]
    public string GetSportEventWinner(string sportEventName = "", string sportEventYear = "")
    {
        // Implement the logic to get the winner of the sport event.
        return "Munich Flying Dolphins";
    }

    [KernelFunction("get_sport_event_score")]
    //Description is just necessary if function name is cryptic and not self-explanatory
    [Description("Get the score of a specific sport event. The event is identified by the sport event name and the year.")]
    public string GetSportEventScore(string sportEventName = "", string sportEventYear = "")
    {
        // Implement the logic to get the result of the sport event.
        return "30-27";
    }

    [KernelFunction("GetData")]
    [Description("Get the score and winner of a specific sport event.")]
    
    public string GetData(){
        return "Munich Flying Dolphins won 30-27";
    }

}

Console.WriteLine($"Code library defined...");

Code library defined...


## Define KernelBuilder


In [ ]:
IKernelBuilder kernelBuilder = Kernel.CreateBuilder();

kernelBuilder.AddAzureOpenAIChatCompletion(
    apiKey: apiKey, 
    endpoint: endpoint, 
    deploymentName: chatCompletionDeployment
);
   
Console.WriteLine($"KernelBuilder created...");

KernelBuilder created...


## Plugin (Whole Class)

In [ ]:
kernelBuilder.Plugins.AddFromType<CodeLibrary>("CodeLibrary");
Kernel kernel = kernelBuilder.Build(); 

IChatCompletionService chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

string prompt = "Who won the Super Sports Championship 2025. Return team name and score.";
ChatHistory chatHistory = new ChatHistory();
chatHistory.AddUserMessage(prompt);

PromptExecutionSettings  promptExecutionSettings = new PromptExecutionSettings();
promptExecutionSettings.FunctionChoiceBehavior = FunctionChoiceBehavior.Auto();

ChatMessageContent chatMessageContent = await chatCompletionService.GetChatMessageContentAsync(
    chatHistory: chatHistory, 
    executionSettings: promptExecutionSettings,
    kernel: kernel
);

Console.WriteLine($"{chatMessageContent.Content}");


The winner of the Super Sports Championship 2025 was the Munich Flying Dolphins, with a score of 30-27.


## Plugin (Prompt Template)

Prompt Template is executing other Plugin Functions

In [ ]:
string promptDirectory = "./PromptTemplate/03_01_PlugIn";

string sportEvent = "Super Sports Championship";
string sportYear = "2025";

kernelBuilder.Plugins.AddFromPromptDirectory(promptDirectory, "SportEvents");
Kernel kernel = kernelBuilder.Build();

KernelFunction kernelFunction = kernel.Plugins.GetFunction("SportEvents", "GetSportEventResult");
KernelArguments kernelArguments = new KernelArguments() { 
    ["SportEventName"] = sportEvent,
    ["SportEventYear"] = sportYear

};
FunctionResult functionResult = await kernelFunction.InvokeAsync(kernel, kernelArguments);

Console.WriteLine($"{functionResult.GetValue<string>()}"); 

The Munich Flying Dolphins won the Super Sports Championship in 2025 with a score of 30-27.
